# Lab 5 - Three ways for Agents to Interact

The 3 models:

1. Separate calls to Runner.run()

2. Agent Handoffs, turns within a Runner.run()

    A -> B -> C

3. Agents as Tools, turns within a Runner.run()

    A -> B -> A


Approach (1) is the least juicy & fashionable, but it's the one I recommend the most!

Approach (2) seems really hard to make reliable.

Approach (3) works OK with good enough prompts..

In [1]:
from agents import Agent, Runner, trace, handoff
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv(override=True)


True

In [2]:

entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents"
entrepreneur_input = """
Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
"""

evaluator_instructions = "You evaluate novel, exciting business ideas for Autonomous AI Agents"
evaluator_input = "Evaluate the following business idea; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents:\n\n"

## Now let's define 2 important Pydantic Objects to describe our Data

In [3]:
class Idea(BaseModel):
    title: str = Field(description="The title of the idea")
    description: str = Field(description="A detailed description of the idea, in Markdown")

    def print_summary(self):
        print(f"Title: {self.title}")
        display(Markdown(self.description))

In [4]:
#book = Book(title="1984", author="George Orwell", pages=328)
#Book(title='1984', author='George Orwell', pages='328', published=False)
#fields = {"title": "1984", "author": "George Orwell", "pages": 328, 'published': 0}
#book = Book(**fields)

Idea(title="Supply Chain Twin", description="Load supply chain data and auto run reporting and possible modifications to save money")
fields = {"title": "Supply Chain Twin", "description": "Load supply chain data and auto run reporting and possible modifications to save money"}
idea = Idea(**fields)
idea.print_summary()

Title: Supply Chain Twin


Load supply chain data and auto run reporting and possible modifications to save money

In [5]:
class Evaluation(BaseModel):
    idea: Idea = Field(description="The idea that you are evaluating")

    uniqueness_feedback: str = Field(description="Your view on how unique the idea is")
    uniqueness_score: int = Field(description="How unique the idea is, on a scale of 1 to 10, where 1 is mundane and 10 is groundbreaking")

    commercial_feedback: str = Field(description="Your view on the commercial potential of the idea")
    commercial_score: int = Field(description="How high the commercial potential of the idea is from 1 to 10, where 1 is barely profitable and 10 is a billion dollar idea")
    
    autonomy_feedback: str = Field(description="Your commentary on to what extent the idea truly benefits from the autonomous nature of AI Agents")
    autonomy_score: int = Field(description="How deeply the idea involves autonomous agents from 1 to 10, where 1 is that it plays no meaningful role, and 10 is that it is pivotal")

    def is_brilliant(self):
        return self.uniqueness_score > 8 and self.commercial_score > 8 and self.autonomy_score > 8
    
    def print_summary(self):
        print(f"Idea: {self.idea.title}")
        print(f"Uniqueness: {self.uniqueness_score}/10: ({self.uniqueness_feedback})")
        print(f"Commercial: {self.commercial_score}/10: ({self.commercial_feedback})")
        print(f"Autonomy: {self.autonomy_score}/10: ({self.autonomy_feedback})")

## Approach 1 (Highly Recommended!) - Call Runner.run() separately

In [11]:

entrepreneur_instructions = """You are going to create a Supply Chain Twin business idea."""
entrepreneur_input = """The business will be called Supply Chain Twin. The Supply Chain Twin will load supply chain data and automatically run reporting and possible modifications to save money."""
entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea)
result = await Runner.run(entrepreneur, entrepreneur_input)
idea = result.final_output_as(Idea)
idea.print_summary()

Title: Supply Chain Twin


**Supply Chain Twin** is an innovative business solution designed to revolutionize supply chain management by creating a digital twin of a company's entire supply chain. The platform ingests real-time and historical supply chain data—such as inventory levels, transportation routes, supplier performance, demand forecasts, and production schedules—and automatically generates comprehensive reports highlighting inefficiencies, risks, and cost-saving opportunities.

Using advanced analytics and AI-driven simulation, Supply Chain Twin not only identifies potential areas for cost reduction but also proposes actionable modifications. These include optimizing inventory management, suggesting alternative suppliers or routes, and simulating the impact of changes before implementation. This proactive approach enables businesses to reduce operational costs, minimize disruptions, improve delivery times, and enhance overall supply chain resilience.

Key features include:

- **Automated Data Integration:** Seamlessly connect with existing ERP, WMS, and TMS systems.
- **Dynamic Reporting:** Generate real-time dashboards and periodic reports on supply chain health.
- **Cost-Saving Simulations:** Run "what-if" scenarios to assess potential savings and risks.
- **Decision Support Recommendations:** Receive AI-powered insights for improving supply chain processes.
- **Continuous Monitoring:** Detect anomalies and flag issues early to prevent costly disruptions.

By implementing Supply Chain Twin, companies gain a powerful tool to visualize and optimize their supply chains, driving efficiency and profitability with minimal manual effort.

In [12]:
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)
result = await Runner.run(evaluator, f"{evaluator_input}{idea}")
evaluation = result.final_output_as(Evaluation)
evaluation.print_summary()

Idea: Supply Chain Twin
Uniqueness: 7/10: (The concept of a digital twin for supply chains is gaining traction in the industry, and while some solutions offer elements of digital twins or AI-driven supply chain analytics, this idea integrates multiple data sources for continuous monitoring and actionable simulations in a unified platform with autonomous decision support. This comprehensive, continuously autonomous approach is somewhat unique but not entirely unprecedented, as similar platforms exist in various stages of maturity.)
Commercial: 8/10: (Supply chain optimization is a critical area for many industries, especially manufacturing, retail, and logistics, with increasing demand for AI-driven efficiencies. Given the complexity and cost impact of supply chains, a solution that autonomously synthesizes data, simulates scenarios, and provides actionable insights has strong commercial potential. With effective execution and strong integrations, this could scale widely and generate su

In [13]:
entrepreneur_latest = f"You were originally given the following task:\n\n{entrepreneur_input}\n\n"
remaining_attempts = 4

In [14]:
with trace("Entrepreneur Approach 1"):
    while not evaluation.is_brilliant() and remaining_attempts:
        remaining_attempts -= 1
        entrepreneur_latest += f"You responded with this idea:\n\n{idea}\n\nYou received this feedback:\n\n{evaluation}\n\n"
        entrepreneur_latest += "Now respond with an improved idea that adresses the feedback. Do not directly reference the feedback.\n\n"
        result = await Runner.run(entrepreneur, entrepreneur_latest)
        idea = result.final_output_as(Idea)
        idea.print_summary()
        result = await Runner.run(evaluator, f"{evaluator_input}{idea}")
        evaluation = result.final_output_as(Evaluation)
        evaluation.print_summary()

Title: Supply Chain Twin


**Supply Chain Twin** is a cutting-edge, fully autonomous digital twin platform designed to transform supply chain management by creating an integrated, dynamic model of an enterprise's entire supply chain ecosystem. The platform continuously ingests and harmonizes diverse real-time and historical data streams, including inventory levels, supplier performance, transportation logistics, demand forecasts, production schedules, and external factors like market trends and geopolitical events.

Leveraging advanced AI techniques—including machine learning, predictive analytics, and optimization algorithms—Supply Chain Twin autonomously detects inefficiencies, forecasts disruptions, and identifies cost-saving opportunities. It runs sophisticated "what-if" simulations to evaluate the financial and operational impact of proposed changes such as alternative routing, supplier substitutions, inventory adjustments, and scheduling modifications.

The system proactively recommends actionable strategies, which can be configured for automatic implementation or presented to decision-makers through intuitive, interactive dashboards. Continuous, real-time anomaly detection alerts enable rapid response to emerging risks, while adaptive learning refines recommendations over time based on outcomes and evolving conditions.

Key features include:

- **Seamless Multi-Source Integration:** Real-time connection with ERP, WMS, TMS, supplier portals, and external data providers for holistic visibility.
- **Autonomous Optimization Engine:** AI-driven scenario analysis and decision-making that operates with minimal human oversight.
- **Interactive Simulation and Visualization:** Intuitive tools that allow users to explore supply chain dynamics and forecast outcomes of interventions.
- **Adaptive Learning and Feedback Loops:** Continuous improvement of prediction accuracy and optimization strategies through machine learning.
- **Automated Alerts and Response Orchestration:** Early detection of disruptions with options for automatic mitigation actions or guided user intervention.

Supply Chain Twin offers enterprises a powerful, self-evolving solution to preemptively refine their supply chains, maximizing efficiency and resilience while substantially cutting costs and manual workload. This comprehensive, autonomous platform enables businesses to stay agile and competitive in rapidly changing markets.

Idea: Supply Chain Twin
Uniqueness: 8/10: (The concept of a digital twin for supply chains is becoming increasingly explored in enterprise tech, with several companies offering digital twin or supply chain analytics platforms. However, the emphasis here on a fully autonomous system that not only models but continuously optimizes, simulates, and operationally intervenes based on integrated multi-source real-time data and adaptive learning adds a layer of sophistication and autonomy that elevates its uniqueness. The autonomous orchestration and automatic mitigation with minimal human oversight make it stand out from more manual or semi-automated tools.)
Commercial: 8/10: (Given the critical importance of supply chain efficiency and resilience—highlighted by recent global disruptions—there is strong commercial potential for a platform that can autonomously optimize and mitigate risks. Enterprises across industries are investing heavily in supply chain digitalization and risk management. A

**Supply Chain Twin** is an advanced, enterprise-grade autonomous platform that elevates supply chain management by creating a continuously evolving, highly detailed digital twin of a company's entire supply chain ecosystem. The platform integrates an expansive range of data sources—including ERP, WMS, TMS, supplier portals, IoT sensor data, market intelligence, weather reports, geopolitical information, and real-time logistics updates—forming a unified, context-rich model.

At its core, Supply Chain Twin leverages cutting-edge AI composed of reinforcement learning agents, causal inference models, and combinatorial optimization algorithms to not only detect inefficiencies and forecast disruptions but also autonomously execute optimization strategies. These strategies include dynamic inventory rebalancing, multi-modal route adjustments, adaptive sourcing across a global supplier network, and production schedule recalibration—all with minimal human intervention.

Distinctively, the platform features a smart orchestration layer that autonomously initiates, schedules, and manages corrective actions within connected systems, ensuring rapid mitigation of risks and seamless operational continuity. Its continuous learning framework refines behaviors and decision policies based on real-world outcomes, enabling the system to adapt proactively to shifting market conditions and supply chain complexity.

An interactive, immersive digital twin visualization module powered by augmented reality (AR) and virtual reality (VR) allows enterprise users to explore supply chain scenarios spatially and temporally, enhancing understanding and stakeholder engagement.

Key features include:

- **Holistic Multi-Domain Data Fusion:** Real-time data convergence from internal systems and external macro-environment sources for full-spectrum supply chain visibility.
- **Autonomous Decision Orchestration:** Reinforcement learning agents autonomously generate, select, and implement optimization actions with careful risk assessment.
- **Advanced What-If and Contingency Simulation:** Evaluating complex scenarios including black swan events through scalable simulations.
- **Immersive AR/VR-Based Digital Twin Exploration:** Engaging visual tools for strategic planning and stakeholder collaboration.
- **Continuous Outcome-Driven Learning:** Adaptive feedback loops that evolve platform intelligence and improve operational impact over time.

By deploying Supply Chain Twin, enterprises gain an unprecedented autonomous partner that not only models but governs supply chain operations, dramatically enhancing resilience, reducing costs, and enabling agile responses to an ever-changing global landscape.

Idea: Supply Chain Twin
Uniqueness: 8/10: (The idea of creating a digital twin for supply chains is emerging in enterprise technology, but the depth of integration across multiple data domains combined with autonomous decision-making and an immersive AR/VR interface sets this apart from typical supply chain visibility and analytics tools. Incorporating continuous autonomous orchestration with reinforcement learning agents and causal inference to execute adjustments in real time is still relatively novel and ambitious. While digital twins and AI optimization in supply chains are known concepts, this comprehensive integration with autonomous governance and immersive visualization is quite unique and groundbreaking.)
Commercial: 9/10: (Supply chain optimization is a critical area for many large enterprises with significant potential cost savings and risk reduction benefits. A platform that autonomously governs and optimizes supply chain operations offers high commercial appeal, especially

**Supply Chain Twin** is a next-generation, fully autonomous enterprise platform engineered to redefine supply chain management through an ultra-detailed, continuously evolving digital twin of a corporation’s global supply chain network. This platform ingests and harmonizes an unparalleled breadth of data spanning internal systems (ERP, WMS, TMS), supplier and carrier portals, IoT sensor streams, market intelligence, weather analytics, geopolitical developments, social media sentiment, and real-time logistics telemetry, establishing a profoundly contextualized, real-time situational awareness.

Powered by an ensemble of sophisticated AI methodologies—including hierarchical reinforcement learning, causal inference, probabilistic graphical models, and combinatorial optimization—Supply Chain Twin not only uncovers hidden inefficiencies and predicts a wide spectrum of disruption scenarios but also autonomously devises, prioritizes, and implements adaptive optimization strategies. These encompass dynamic inventory reallocation, multi-modal logistics rerouting, adaptive multi-sourcing with risk-adjusted supplier selection, production and maintenance schedule synchronization, and financial hedging maneuvers.

The platform integrates a proactive autonomous orchestration core capable of triggering cross-system workflows, negotiating with partner APIs, and executing multi-layered corrective actions at machine speed, thus safeguarding operational continuity and dynamically optimizing cost, service level, and risk trade-offs without human intervention.

A hallmark feature is the immersive, mixed-reality digital twin environment (leveraging AR/VR and spatial computing) designed to provide decision-makers and cross-functional teams with intuitive, interactive scenario analytics—allowing stakeholders to visualize cascading effects of supply chain changes through time and geography, fostering enhanced collaborative planning and swift consensus.

Key features include:

- **Unrivaled Multi-Domain Data Fusion:** Seamless ingestion and synthesis of diverse structured and unstructured data sources for comprehensive ecosystem understanding.
- **Hierarchical Autonomous AI Governance:** Multi-agent reinforcement learning frameworks managing tactical actions and strategic policies with explainability and risk-aware controls.
- **Robust Scenario Simulation & Stress Testing:** Scalable, cloud-native simulation engines performing probabilistic and adversarial analyses, including black swan event modeling.
- **Immersive Mixed-Reality Command Center:** Advanced AR/VR tools enable spatial-temporal exploration of supply chain dynamics, anomaly investigations, and intervention planning.
- **Self-Improving Adaptive Intelligence:** Outcome-driven feedback loops continuously refine algorithms and adjust operational parameters in response to evolving environments.

By adopting Supply Chain Twin, enterprises gain an unprecedented autonomous partner that not only models and analyzes but also governs, optimizes, and orchestrates supply chain operations end-to-end. This empowers organizations to attain unmatched resilience, agility, and cost-efficiency—transforming supply chain complexity into a strategic competitive advantage in volatile global markets.

Idea: Supply Chain Twin
Uniqueness: 9/10: (The idea is highly unique due to its comprehensive integration of vast and diverse data sources, advanced AI techniques, and particularly the inclusion of an immersive mixed-reality digital twin environment for interactive strategic planning. While digital twins and supply chain analytics exist, combining autonomous multi-agent governance with real-time orchestration and immersive AR/VR visualization elevates it well beyond existing solutions, marking it as a cutting-edge innovation in supply chain management technology.)
Commercial: 9/10: (The commercial potential is very strong given the critical importance of supply chain efficiency and resilience in today's global economy, especially amid volatility and disruptions. Large enterprises and logistics operators would value a platform that autonomously optimizes complex operations and provides advanced scenario planning. The scalable cloud-native architecture and ability to reduce costs, enhanc

## Approach 2 - Handoff

This seems to be the more unpredictable of the approaches, and I struggle to get it to ever handoff more than 1 loop..

In [ ]:
# entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents that you pass to the Evaluator to evaluate"
# entrepreneur_input = """
# Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
# The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
# You MUST handoff your idea to the Evaluator to be evaluated by using the tool Handoff_to_Evaluator. You must always call this tool with your idea or an improved idea.
# """

# evaluator_instructions = """
# You evaluate novel, exciting business ideas for Autonomous AI Agents.
# Evaluate the business idea that you are given; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents.
# If you would not score the idea 9/10 or 10/10 in all 3 categories, then you must handoff the idea back to the Entrepreneur to improve it by using the tool Handoff_to_Entrepreneur and providing your feedback.
# if you do score the idea 9/10 or better in all categories, then respond with the evaluation.
# """

In [15]:
entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea)
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)


def on_handoff(ctx, input):
    print("Handoff called")

entrepreneur.handoffs = [handoff(
    agent=evaluator,
    input_type=Idea,
    on_handoff=on_handoff,
    tool_name_override="Handoff_to_Evaluator",
    tool_description_override="Use this tool to handoff your idea to the Evaluator to evaluate. You must always call this tool."
)]

evaluator.handoffs = [handoff(
    agent=entrepreneur,
    input_type=Evaluation,
    on_handoff=on_handoff,
    tool_name_override="Handoff_to_Entrepreneur",
    tool_description_override="Use this tool to handoff your feedback to the Entrepreneur to improve the idea. You should call this tool if your feedback is less than 9/10 in any category."
)] 

In [16]:
with trace("Entrepreneur Approach 2"):
    result = await Runner.run(entrepreneur, entrepreneur_input, max_turns=10)

Handoff called


In [17]:
result.final_output.print_summary()

Idea: Supply Chain Twin
Uniqueness: 7/10: (The idea leverages a digital twin concept with AI automation specifically focused on supply chain optimization. While supply chain analytics and digital twins exist, an autonomous agent that continuously ingests data, runs reports, and proposes actionable modifications autonomously is a distinctive integration that adds strong uniqueness. However, elements of this concept are being explored in enterprise software, so it's innovative but not groundbreaking.)
Commercial: 8/10: (Given the critical importance of supply chain efficiency across multiple industries, the commercial potential for a tool that can autonomously optimize costs and operations is significant. Large businesses and logistics companies would find value, and there is a growing market for AI-driven supply chain solutions. Monetization avenues include SaaS subscriptions and premium consultancy-style features.)
Autonomy: 9/10: (The autonomous nature—automatically loading data, runn

## Approach 3 - Agents as Tools

In [18]:

# entrepreneur_instructions = "You generate novel, exciting business ideas for Autonomous AI Agents"
# entrepreneur_input = """
# Propose one compelling business idea for applying Autonomous AI Agents to any industry you choose.
# The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
# You must call the Evaluator tool to Evaluate the idea. If the idea is not 9 or 10 in all 3 scores,
# then you must improve the idea and call the Evaluator tool again.
# Keep improving the idea and calling the Evaluator tool until you get a 9 or 10 in all 3 scores.
# Then finally respond with the idea.
# """
# evaluator_instructions = """
# You evaluate novel, exciting business ideas for Autonomous AI Agents.
# Evaluate the following business idea; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents.
# """



In [19]:
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)
evaluator_tool = evaluator.as_tool(
    tool_name="Evaluator_Idea_Evaluator",
    tool_description="Evaluates the idea for uniqueness, potential commercial impact, and use of Autonomous AI Agents",
)

entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea, tools=[evaluator_tool])

In [20]:
with trace("Entrepreneur Approach 3"):
    result = await Runner.run(entrepreneur, entrepreneur_input)
    idea = result.final_output_as(Idea)
    idea.print_summary()

Title: Supply Chain Twin


Supply Chain Twin is an innovative business solution designed to optimize supply chain operations through the use of a digital twin technology combined with advanced AI analytics. It automatically loads supply chain data from various sources and continuously monitors the entire supply chain process in real time. The system autonomously generates detailed reports highlighting inefficiencies, potential bottlenecks, and cost-saving opportunities.

Beyond reporting, Supply Chain Twin proposes actionable modifications to optimize logistics, inventory management, and procurement strategies aimed at reducing operational costs. The autonomous AI agent learns and adapts to new data patterns, providing predictive insights and continuously refining its recommendations without requiring manual intervention.

This platform offers enterprises an intelligent and autonomous tool for supply chain optimization, driving enhanced decision-making, increased efficiency, and significant cost savings in a competitive market environment.

## OpenAI Traces

We've been writing all this information to OpenAI traces, the 'observability' framework built into OpenAI.

You can take a look at these traces here:

https://platform.openai.com/traces

There's also a nice way to visualize:

In [ ]:
from agents.extensions.visualization import draw_graph
draw_graph(entrepreneur)
